# 🗺️ Partie 6 : Visualisation & Recommandation

## 🎯 Objectif
Présenter les résultats finaux sous forme de tableau de bord interactif pour l'utilisateur final ou l'équipe marketing.

## 🛠 Choix Techniques
* **Plotly Express (`scatter_mapbox`)** : Choix d'une librairie interactive (zoom, survol) plutôt que statique (Matplotlib) pour une meilleure expérience utilisateur sur des données géographiques.
* **Double Visualisation** :
    1.  **Top 5 Destinations** : Basé purement sur le score météo.
    2.  **Top 20 Hôtels** : Filtrage des meilleurs établissements situés *uniquement* dans les zones recommandées.

In [1]:
import pandas as pd
import plotly.express as px

# Chargement des données (Simulation d'une lecture depuis le Data Warehouse ou CSV locaux)
# Dans un contexte de production, on ferait un `pd.read_sql` ici.
df_weather = pd.read_csv('cities_weather_data_7days.csv')
df_hotels = pd.read_csv('hotels_data.csv')

# --- PARTIE 1 : ANALYSE MÉTÉO ---
# On trie par température (descendant) pour avoir les villes les plus chaudes
# On pourrait aussi pondérer avec la pluie (ex: score = temp - (pluie * 2))
top_5_cities = df_weather.sort_values(by='avg_temp_7_days', ascending=False).head(5)

print("🏆 Top 5 Destinations (Météo) :")
print(top_5_cities[['city_name', 'avg_temp_7_days']])

# Carte 1 : Météo
fig_weather = px.scatter_mapbox(
    top_5_cities, 
    lat="latitude", 
    lon="longitude",
    color="avg_temp_7_days",
    size="avg_temp_7_days",
    color_continuous_scale=px.colors.sequential.Bluered,
    size_max=20,
    zoom=4.5,
    mapbox_style="open-street-map",
    title="Top 5 Destinations de la Semaine (Basé sur la météo)",
    hover_name="city_name",
    hover_data=["total_rain_7_days", "main_weather"]
)
fig_weather.show()

# --- PARTIE 2 : RECOMMANDATION HÔTELS ---
# Filtrage : On ne garde que les hôtels situés dans nos Top 5 villes
best_cities_list = top_5_cities['city_name'].tolist()
df_hotels_filtered = df_hotels[df_hotels['city'].isin(best_cities_list)].copy()

# Nettoyage et conversion du score en numérique
df_hotels_filtered['score'] = pd.to_numeric(df_hotels_filtered['score'], errors='coerce')

# Sélection des 20 meilleurs hôtels dans ces zones
top_20_hotels = df_hotels_filtered.sort_values(by='score', ascending=False).head(20)

# Pour la carte, on a besoin des coordonnées. 
# Simplification : On utilise les coordonnées de la ville (dispo dans df_weather)
df_merged = top_20_hotels.merge(df_weather[['city_name', 'latitude', 'longitude']], 
                                left_on='city', right_on='city_name')

print(f"\n🏨 Sélection des 20 meilleurs hôtels dans les zones : {best_cities_list}")

# Carte 2 : Hôtels
fig_hotels = px.scatter_mapbox(
    df_merged, 
    lat="latitude", 
    lon="longitude",
    color="score",
    size="score",
    color_continuous_scale=px.colors.sequential.Viridis, # Vert = Bien
    size_max=15,
    zoom=5,
    mapbox_style="open-street-map",
    title="Top 20 Hôtels dans les Meilleures Zones Météo",
    hover_name="hotel_name",
    hover_data=["city", "score", "url"]
)
fig_hotels.show()

🏆 Top 5 Destinations (Météo) :
                   city_name  avg_temp_7_days
20                 Marseille            13.04
27                 Collioure            12.38
19                    Cassis            12.08
26  Saintes Maries de la mer            11.65
21           Aix en Provence            11.55


C:\Users\Athanor PC\AppData\Local\Temp\ipykernel_33448\2714231580.py:18: DeprecationWarning: *scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/
  fig_weather = px.scatter_mapbox(



🏨 Sélection des 20 meilleurs hôtels dans les zones : ['Marseille', 'Collioure', 'Cassis', 'Saintes Maries de la mer', 'Aix en Provence']


C:\Users\Athanor PC\AppData\Local\Temp\ipykernel_33448\2714231580.py:53: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



## 🏆 Conclusion du Projet

Ce projet a permis de construire un pipeline de données complet (**End-to-End Data Engineering**) répondant à une problématique business concrète.

**Points clés validés :**
* ✅ **Collecte de données hétérogènes** : API structurée (Météo) et Web Scraping complexe (Booking).
* ✅ **Infrastructure Cloud Hybride** : Utilisation conjointe d'un Data Lake (S3) pour le stockage brut et d'un Data Warehouse (RDS SQL) pour la donnée structurée.
* ✅ **Robustesse** : Implémentation de mécanismes de sécurité (Variables d'environnement) et de résilience (Retry logic).
* ✅ **Valorisation** : Livraison d'un outil de visualisation interactif aide à la décision.